## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-16-05-44-39 +0000,bbc,Nigerian chef breaks world record for largest ...,https://www.bbc.com/news/articles/cz7r0rjzpqgo...
1,2025-09-16-05-42-28 +0000,nypost,TV anchor Beni Rae Harmony resigns after being...,https://nypost.com/2025/09/16/us-news/illinois...
2,2025-09-16-05-40-22 +0000,bbc,Israel escalates bombardment as defence minist...,https://www.bbc.com/news/articles/c749q1245pwo...
3,2025-09-16-05-31-26 +0000,nypost,Visa revocations already underway for foreigne...,https://nypost.com/2025/09/16/us-news/visa-rev...
4,2025-09-16-05-27-10 +0000,wapo,Trump files $15 billion defamation and libel l...,https://www.washingtonpost.com/business/2025/0...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2301/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
41,trump,56
21,charlie,31
22,kirk,31
48,new,19
98,china,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
57,2025-09-16-00-41-00 +0000,wsj,President Trump got a win when the Senate conf...,https://www.wsj.com/economy/central-banking/st...,128
45,2025-09-16-01-14-00 +0000,wsj,A federal appeals court on Monday night reject...,https://www.wsj.com/economy/central-banking/ap...,111
58,2025-09-16-00-39-31 +0000,nypost,Senate confirms Trump Fed pick Stephen Miran a...,https://nypost.com/2025/09/15/us-news/senate-c...,111
11,2025-09-16-04-22-08 +0000,nypost,Old man who falsely claimed he shot Charlie Ki...,https://nypost.com/2025/09/16/us-news/old-man-...,110
7,2025-09-16-04-57-35 +0000,bbc,Trump says he will sue New York Times for $15bn,https://www.bbc.com/news/articles/cevz419pk8vo...,109


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
57,128,2025-09-16-00-41-00 +0000,wsj,President Trump got a win when the Senate conf...,https://www.wsj.com/economy/central-banking/st...
11,110,2025-09-16-04-22-08 +0000,nypost,Old man who falsely claimed he shot Charlie Ki...,https://nypost.com/2025/09/16/us-news/old-man-...
45,55,2025-09-16-01-14-00 +0000,wsj,A federal appeals court on Monday night reject...,https://www.wsj.com/economy/central-banking/ap...
128,52,2025-09-15-20-29-55 +0000,cbc,"Carney, Poilievre come face-to-face for the fi...",https://www.cbc.ca/news/politics/carney-poilie...
87,47,2025-09-15-22-46-10 +0000,nyt,U.S.-China Deal to Avert TikTok Ban May be Clo...,https://www.nytimes.com/2025/09/15/technology/...
136,41,2025-09-15-20-15-10 +0000,nypost,Trump urges Republicans to back stopgap bill a...,https://nypost.com/2025/09/15/us-news/trump-ca...
217,40,2025-09-15-15-15-00 +0000,wsj,Canadian factories notched the biggest rise in...,https://www.wsj.com/articles/canada-factory-an...
225,39,2025-09-15-14-39-54 +0000,nypost,Tesla share surge as Elon Musk buys $1B in sto...,https://nypost.com/2025/09/15/business/tesla-s...
150,39,2025-09-15-19-27-18 +0000,nypost,"Russia is ‘losing’ its war on Ukraine, Joint S...",https://nypost.com/2025/09/15/world-news/russi...
175,35,2025-09-15-17-50-03 +0000,wapo,"Rubio downplays U.S. tensions with Israel, pla...",https://www.washingtonpost.com/national-securi...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
